# Minimal code for Extended Data Figure 1a flow cytometry panel

In [ ]:
# library for reading .fcs files
suppressPackageStartupMessages(library(flowCore))
# library for extracting statistical fits
suppressPackageStartupMessages(library(broom))
# standard analysis and plotting functions, includes dplyr, ggplot2
suppressPackageStartupMessages(library(tidyverse))
# loads lab default ggplot2 theme and provides color-blind friendly palette
suppressPackageStartupMessages(library(rasilabRtemplates))
# this folder contains the data and annotations
fcs_file_folder <- c("../../../../data/flow_cytometry/figs1a_eyfp_reporter_sgeyfp_u2os_293t/day1/", 
                     "../../../../data/flow_cytometry/figs1a_eyfp_reporter_sgeyfp_u2os_293t/day3/", 
                     "../../../../data/flow_cytometry/figs1a_eyfp_reporter_sgeyfp_u2os_293t/day5/",
                     "../../../../data/flow_cytometry/figs1a_eyfp_reporter_sgeyfp_u2os_293t/day8/",
                     "../../../../data/flow_cytometry/figs1a_eyfp_reporter_sgeyfp_u2os_293t/day11/")
# nice channel names
channels <- c(
  "fitc_a" = "yfp", "pe_texas_rd_a" = "rfp",
  "bv421_a" = "bfp"
)

In [ ]:
if (!file.exists('../figures')){
    dir.create('../figures')
}

## Read in annotations

In [ ]:
annotations <- read_csv("../annotations/sample_annotations.csv") %>% 
  mutate(file = as.character(file)) %>%
  print()

## Read in flow data

In [ ]:
options(warn=-1)

flowdata  <- list.files(fcs_file_folder, full.names = T, recursive = T, pattern = ".fcs") %>% 
  as_tibble_col("file") %>%
  mutate(day = as.integer(str_extract(file, "(?<=day)\\d+"))) %>% 
  mutate(data = map(file, . %>% read.FCS(transformation = F, alter.names = T) %>% exprs %>% as_tibble)) %>%
  mutate(file = str_extract(file, "(?<=events_)[:graph:]+(?=.fcs$)")) %>%
  unnest("data") %>%
  janitor::clean_names() %>%
  rename_with(. %>% channels[.], names(channels)) %>%
  select(day, file, ssc_a, bfp, yfp, rfp) %>%
  print()

## Join data with annotations

In [ ]:
data <- flowdata %>% 
     inner_join(annotations, by = "file") %>% 
     print() 

## Plot YFP expression over time

In [ ]:
options(repr.plot.width = 5.5, repr.plot.height = 2.5)

plot_data <- data %>%
  # collapse all channels
  pivot_longer(c("bfp", "yfp", "rfp", "ssc_a"), names_to = "channel") %>% 
  filter(channel %in% c("yfp")) %>% 
  mutate(sgRNA = case_when(
    sgRNA == "parent" ~ "Parent",
    sgRNA == "FLUC" ~ "sgCTRL",
    sgRNA == "YFP" ~ "sgEYFP",
  )) %>%
  # filter(sgrna == "sgEYFP") %>% 
  mutate(channel = case_when(
    channel == "yfp" ~ "EYFP",
  )) %>%
  filter(value >= 1) %>%
  print()

plot_data %>%
  ggplot(aes(x = value, y = as.factor(day), fill = sgRNA)) +
  ggridges::geom_density_ridges(alpha = 0.8) +
  facet_wrap(~cell_line) +
  scale_x_log10(limits = c(1, 1e6), breaks = scales::trans_breaks("log10", function(x) 100^x),
  labels = scales::trans_format("log10", scales::math_format(10^.x))) +
  # annotation_logticks(sides = "b") +
  scale_fill_manual(values = cbPalette[c(1,3,2)]) +
  labs(x = "Fluorescence (a.u.)", y = "Days post Cas9", fill = "") +
  theme(axis.line = element_line(color = "grey"),
  axis.ticks = element_line(color = "grey")) +
  NULL
  
ggsave("../figures/sgyfp_sgfluc_effects_for_validation.pdf", width = 5.5, height = 2.5)